# Example 2: Nonlinear Time-History Analysis of MDOF Systems using the "modeller" Class

## Introduction

This Jupyter Notebook provides a structured workflow for performing nonlinear time-history analyses (NLTHA) on multi-degree-of-freedom (MDOF) structural models. By combining functions for MDOF modeling and dynamic analysis, the notebook enables the setup, execution, and post-processing of structural responses under earthquake loading.

The key objectives of this notebook are:

1. **SDoF-to-MDoF Model Calibration**: Calibrate storey-based force-deformation relationships using SDoF capacity curve definition (spectral displacement-spectral acceleration) based on the methodology of Lu et al. (2020) and other modifications to account for distinct response typologies (i.e., bilinear, trilinear and quadrilinear backbone definitions)

2. **MDOF Model Construction**: Define and assemble MDOF models by specifying essential structural properties, including:
   - Mass, heights, fundamental period, etc.
   - Nonlinear response characteristics at each degree of freedom

3. **Nonlinear Time-History Analysis (NLTHA)**: Simulate the dynamic response of MDOF structures under time-dependent inputs, such as ground motion records, to realistically assess structural behavior and response metrics (e.g., peak storey drifts, peak floor accelerations) under loading conditions and extract critical response metrics and model information.

The notebook provides a step-by-step guide, covering each phase from MDOF model calibration, setup to input parameter configuration, analysis execution, and detailed results extraction. Users should have some familiarity with python scripts, structural dynamics and computational modeling to fully benefit from this material.

---

## Prerequisites

To run this notebook successfully, the following "non-native" Python packages are required:
-  openseespy: A Python library for performing finite element analysis based on the OpenSees framework.
    - Reference: https://openseespydoc.readthedocs.io/en/latest/

---


## Workflow Overview

1. **Input Parameters** 
2. **Initialize Libraries and Modules**: Load essential libraries and custom functions for MDOF modeling.
3. **Define Directories**: Assign and create directories to import and export data
4. **Import Data**: Automatic import of necessary data
4. **Define Structural Properties**: Set up mass, stiffness, damping, and nonlinear properties for each degree of freedom.
5. **Execute Nonlinear THA**: Run the time-history analysis using dynamic loading inputs.
6. **Post-Process and Visualize Results**: Generate plots and summaries to examine the structure’s response to the applied loading.

By the end of this notebook, users will have a complete, adaptable script for nonlinear dynamic analyses on MDOF structures, supporting a range of investigation scenarios and performance assessments.

Let’s begin by defining initial input parameters and loading the required libraries and setting up the initial parameters for the MDOF model.

# Input Parameters

In [ ]:
### [REQUIRED INPUT] Define the main directory
main_directory = 'C:/Users/m.nafeh/Documents/GitHub/vulnerability-toolkit' # Replace this line with the directory you cloned the scripts to

### [REQUIRED INPUT] Building Class from Database
currentBuildingClass = 'CR_LFINF+CDN+DNO_H2' # The example building class (DO NOT EDIT UNLESS YOU HAVE MADE CHANGES TO INPUT FILES)

# Initialize Libraries and Modules

In [ ]:
import time
import sys

# Import vulnerability-toolkit libraries
sys.path.insert(1, f'{main_directory}/src')
from modeller import *
from calibration import *
from utilities import *
from units import *
from postprocessor import *

start = time.time()

# Define Directories

In [ ]:
# Define the directory of the capacities
capDir= f'{main_directory}/example/in/capacity'          

# Define the directory of the ground-motion records
gmDir  = f'{main_directory}/example/in/records'            

# Define the directory of the damping model
dampDir = f'{main_directory}/example/in/damping'

# Define the directory of the damping model
threshDir = f'{main_directory}/example/in/thresholds'

# Define the main output directory
nlthaOutDir = f'{main_directory}/example/out/nltha'  
if not os.path.exists(f'{nlthaOutDir}'):
    os.makedirs(f'{nlthaOutDir}')

# Import Data

In [ ]:
# Import the building classes dataframe with properties
class_info_df   = pd.read_csv(f'{capDir}/in_plane_capacity_parameters_table.csv')    

# Import the damping model
damp_info_df    = pd.read_csv(f'{dampDir}/global_damping_model.csv')

# Analysis

## Analysis Part 1: SDoF-to-MDoF Calibration

#### The calibration function (calibrateModel) requires six input arguments:
1. Number of storeys
2. First-mode transformation factor (gamma)
3. The capacity array of the single degree-of-freedom oscillator
4. The fundamental period of the single degree-of-freedom oscillator
5. Boolean flag whether the lateral load-resisting system for the considered building class is moment-resisting frames (or not)
6. Boolean flag whether the building class expects a soft-storey mechanism to be activated (or not)

In [ ]:
# 1.) Extract the number of storeys 
number_storeys = class_info_df['Number_storeys'].loc[class_info_df['Building_class']==currentBuildingClass].item()             # Number of Storeys

# 2.) Extract the first-mode transformation factor 
gamma          = class_info_df['Participation_factor'].loc[class_info_df['Building_class']==currentBuildingClass].item()       # Participation factor (gamma) 

# 3.) Import the equivalent SDOF capacity array (without the initial zeros)
sdof_capacity  = np.array(pd.read_csv(f'{capDir}/{currentBuildingClass}.csv', header = None))[1:,:]                            # SDOF capacity array (spectral displacement, spectral acceleration)

# 4.) Extract the fundamental period of the single degree-of-freedom oscillator
sdof_period    = class_info_df['T1'].loc[class_info_df['Building_class']==currentBuildingClass].item()                         # Fundamental period of the equivalent SDoF system 

# 5-6.) Detect whether current model has infills or potential soft-storey mechanism
if 'LFM' in currentBuildingClass or 'LFBR' in currentBuildingClass:
    isFrame = True
else:
    isFrame = False
if 'SOS' in currentBuildingClass:
    isSOS = True
else:
    isSOS = False

#### The calibration function (calibrate_model) from "calibration" returns four output variables:
1. The floor mass array to be assigned to the MDOF model generator (floor_masses)
2. The storey deformation (in m) capacity to be assigned to the MDOF model generator (storey_disps)
3. The acceleration capacity (in g) to be assigned to the MDOF model generator (storey_forces)
4. The considered mode shape (mdof_phi)

In [ ]:
# Calibrate the model using the Lu et al. (2016) method
floor_masses, storey_disps, storey_forces, mdof_phi = calibrate_model(number_storeys, gamma, sdof_capacity, sdof_period, isFrame, isSOS)

print('The mass of each floor (in tonnes):', floor_masses)
print('The first-mode shape used for calibration:', mdof_phi)

# Plot the capacities to visualise the outcome of the calibration
for i in range(storey_disps.shape[0]):
   plt.plot(np.concatenate(([0.0], storey_disps[i,:])), np.concatenate(([0.0], storey_forces[i,:]*9.81)), label = f'Storey #{i+1}')
plt.plot(np.concatenate(([0.0], sdof_capacity[:,0])), np.concatenate(([0.0], sdof_capacity[:,1]*9.81)), label = f'SDOF Capacity')
plt.xlabel('Storey Deformation [m]', fontsize= FONTSIZE_1)
plt.ylabel('Storey Shear [kN]', fontsize = FONTSIZE_1)
plt.legend(loc = 'lower right')
plt.grid(visible=True, which='major')
plt.grid(visible=True, which='minor')
plt.xlim([0.00, 0.03])
plt.show()


## Analysis Part 2: Setting Up Analysis and Running

In [ ]:
# Initialise MDOF storage lists
mdof_coll_index_list = []               # List for collapse index
mdof_peak_disp_list  = []               # List for peak floor displacement (returns all peak values along the building height)
mdof_peak_drift_list = []               # List for peak storey drift (returns all peak values along the building height)
mdof_peak_accel_list = []               # List for peak floor acceleration (returns all peak values along the building height)
mdof_max_peak_drift_list = []           # List for maximum peak storey drift (returns the maximum value) 
mdof_max_peak_drift_dir_list = []       # List for maximum peak storey drift directions
mdof_max_peak_drift_loc_list = []       # List for maximum peak storey drift locations
mdof_max_peak_accel_list = []           # List for maximum peak floor acceleration (returns the maximum value)
mdof_max_peak_accel_dir_list = []       # List for maximum peak floor acceleration directions 
mdof_max_peak_accel_loc_list = []       # List for maximum peak floor acceleration locations 

# Define directory for temporary analysis outputs
nrha_outdir = f'{nlthaOutDir}/{currentBuildingClass}'
if not os.path.exists(f'{nrha_outdir}'):
    os.makedirs(f'{nrha_outdir}')

In [ ]:
# Loop over ground-motion records, compile MDOF model and run NLTHA
gmrs = sorted_alphanumeric(os.listdir(f'{gmDir}/acc'))                         # Sort the ground-motion records alphanumerically
dts  = sorted_alphanumeric(os.listdir(f'{gmDir}/dts'))                         # Sort the ground-motion time-step files alphanumerically

# Extract the building class info necessary to initialise the model generator and analysis
storey_heights       = class_info_df['Storey_height'].loc[class_info_df['Building_class']==currentBuildingClass].item()              # Storey height
floor_heights        = [storey_heights]*number_storeys                                                                               # Create list of heights for MDoF oscillator
mdof_damping         = damp_info_df['Damping_value'].loc[damp_info_df['Building_class']==currentBuildingClass].item()                # Inherent damping value (xi)                  

for i in range(len(gmrs)):

    ### Compile the MDOF model    
    model = modeller(number_storeys,floor_heights,floor_masses,storey_disps,storey_forces*units.g)    # Initialise the class (Build the model)
    model.mdof_initialise()                                                                           # Initialise the domain
    model.mdof_nodes()                                                                                # Construct the nodes
    model.mdof_fixity()                                                                               # Set the boundary conditions 
    model.mdof_material()                                                                             # Assign the nonlinear storey material
    if i==0:
        model.plot_model()                                                                            # Visualise the model
    model.do_gravity_analysis()                                                                       # Do gravity analysis
    T, _ = model.do_modal_analysis(num_modes = number_storeys)                                        # Do modal analysis and get period of vibration

    ### Define ground motion objects
    fnames = [f'{gmDir}/acc/gmr_{i}.csv']                                       # Ground-motion record names
    fdts = f'{gmDir}/dts/dts_{i}.csv'                                           # Ground-motion time-step names 
    dt_gm = pd.read_csv(fdts)[pd.read_csv(fdts).columns[0]].loc[0]              # Ground-motion time-step
    t_max = pd.read_csv(fdts)[pd.read_csv(fdts).columns[0]].iloc[-1]            # Ground-motion duration
   
    ### Define analysis params and do NLTHA
    dt_ansys = dt_gm                                                            # Set the analysis time-step
    sf = units.g                                                                # Set the scaling factor (if records are in g, a scaling factor of 9.81 m/s2 must be used to be consistent with opensees) 
    control_nodes, coll_index, peak_drift, peak_accel, max_peak_drift, max_peak_drift_dir, max_peak_drift_loc, max_peak_accel, max_peak_accel_dir, max_peak_accel_loc, peak_disp = model.do_nrha_analysis(fnames, 
                                                                                                                                                                                                                                   dt_gm, 
                                                                                                                                                                                                                                   sf, 
                                                                                                                                                                                                                                   t_max, 
                                                                                                                                                                                                                                   dt_ansys,
                                                                                                                                                                                                                                   nrha_outdir, 
                                                                                                                                                                                                                                   xi = mdof_damping)

    ### Store the analysis
    mdof_coll_index_list.append(coll_index)
    mdof_peak_drift_list.append(peak_drift)
    mdof_peak_accel_list.append(peak_accel)
    mdof_peak_disp_list.append(peak_disp)
    mdof_max_peak_drift_list.append(max_peak_drift)
    mdof_max_peak_drift_dir_list.append(max_peak_drift_dir)
    mdof_max_peak_drift_loc_list.append(max_peak_drift_loc)
    mdof_max_peak_accel_list.append(max_peak_accel)
    mdof_max_peak_accel_dir_list.append(max_peak_accel_dir)
    mdof_max_peak_accel_loc_list.append(max_peak_accel_loc)

print('ANALYSIS COMPLETED')

## Analysis Part 3: Export The Results

In [ ]:
# Store the analysis results in a dictionary
ansys_dict = {}
labels = ['T','control_nodes', 'mdof_coll_index_list',
          'mdof_peak_drift_list','mdof_peak_accel_list',
          'mdof_max_peak_drift_list', 'mdof_max_peak_drift_dir_list', 
          'mdof_max_peak_drift_loc_list','mdof_max_peak_accel_list',
          'mdof_max_peak_accel_dir_list','mdof_max_peak_accel_loc_list',
          'mdof_peak_disp_list']
for i, label in enumerate(labels):
    ansys_dict[label] = vars()[f'{label}']
# Export the analysis output variable to a pickle file using the "export_to_pkl" function from "utilities"
export_to_pkl(f'{nlthaOutDir}/analysis_{currentBuildingClass}.pkl', ansys_dict) 

In [ ]:
end = time.time()
print('Elapsed Time:', (end - start)/60, 'minutes')